In [1]:
import torch
bart = torch.hub.load('pytorch/fairseq', 'bart.large')
bart.eval()

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_main
2023-11-29 01:03:04.683833: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 01:03:04.725285: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 01:03:05.762556: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


running build_ext


/opt/conda/lib/python3.10/site-packages/torch/utils/cpp_extension.py:502: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
tokens = bart.encode('Hello the fucking world!')
# assert tokens.tolist() == [0, 31414, 232, 328, 2]
bart.decode(tokens)  # 'Hello world!'

In [ ]:
# Extract the last layer's features
last_layer_features = bart.extract_features(tokens)
# assert last_layer_features.size() == torch.Size([1, 5, 1024])
print(last_layer_features.size())

# Extract all layer's features from decoder (layer 0 is the embedding layer)
all_layers = bart.extract_features(tokens, return_all_hiddens=True)
print(all_layers)
assert len(all_layers) == 13
assert torch.all(all_layers[-1] == last_layer_features)

In [ ]:
last_layer_features.shape

In [ ]:
bart.fill_mask(['The cat <mask> on the <mask>.'], topk=3, beam=10)


In [ ]:
!pip install fairseq

In [2]:
from transformers import BartForConditionalGeneration, BartTokenizer


In [10]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large", forced_bos_token_id=0)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")


In [30]:
a = model.model.shared(tokenizer(['a'], return_tensors="pt")['input_ids'])

In [45]:
print(torch.cat([a,a], dim=1).shape)

torch.Size([1, 6, 1024])


In [46]:
model(inputs_embeds=torch.cat([a,a], dim=1), decoder_input_ids=tokenizer(['a'], return_tensors="pt")['input_ids'])

Seq2SeqLMOutput(loss=None, logits=tensor([[[ 22.6671,  -0.8594,  19.5769,  ...,  -0.6536,  -1.2582,   9.5516],
         [-11.1314,  -3.2188,  12.6956,  ...,  -2.7312,  -2.8604,  -2.0370],
         [  6.1925,  -1.8686,  17.7797,  ...,  -2.4432,  -2.2719,   5.1874]]],
       grad_fn=<AddBackward0>), past_key_values=((tensor([[[[ 1.1687,  2.8813,  4.4058,  ..., -1.6669,  3.0377, -2.8006],
          [-1.5492, -2.1931, -2.0090,  ...,  3.3555, -0.1741,  1.7948],
          [-2.5676, -2.6669, -4.6882,  ..., -1.6383, -2.6659, -2.1031]],

         [[-1.3328, -4.4495,  1.5480,  ..., -1.8666,  0.8618, -0.2465],
          [-0.5902,  3.2732, -2.7884,  ...,  2.2697, -0.3216,  0.1803],
          [-3.3394, -3.4319,  1.0716,  ..., -4.8966, -1.3012,  4.3711]],

         [[ 0.1951,  0.9602,  0.9443,  ..., -0.1951, -1.0370,  2.6584],
          [ 0.0982, -0.4285, -0.4920,  ...,  0.6346,  1.2430, -0.1824],
          [ 0.7811,  1.6962,  2.5301,  ..., -1.8475, -0.5803, -0.0753]],

         ...,

         [[ 0.

In [28]:
tokenizer(['a'], return_tensors="pt")

{'input_ids': tensor([[  0, 102,   2]]), 'attention_mask': tensor([[1, 1, 1]])}